# NEURAL NETWORK

## DOCUMENT PREAMBLE

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from tqdm import tqdm
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Configure matplotlib
# tqdm.pandas()
plt.style.use("classic")
#plt.rcParams["figure.dpi"] = 200
plt.rcParams["figure.facecolor"] = "white"
plt.rcParams["font.family"] = "serif"

In [2]:
# Set document parameters
data_version = "demo"
data_type = "train"

In [ ]:
# Load data from parquet files
def load_data(data_version, data_type, print_info=False):
    if data_type not in ["train", "validation"]:
        raise ValueError("data_type must be either 'train' or 'validation'")

    # Read parquet files into DataFrames
    behaviors_df = pd.read_parquet(
        f"./data/ebnerd_{data_version}/{data_type}/behaviors.parquet"
    )
    history_df = pd.read_parquet(f"./data/ebnerd_{data_version}/{data_type}/history.parquet")
    articles_df = pd.read_parquet(f"./data/ebnerd_{data_version}/articles.parquet")

    # Print DataFrame info
    if print_info:
        for name, df in zip(
            [f"{data_type}/behaviors", f"{data_type}/history", "articles"],
            [behaviors_df, history_df, articles_df],
        ):
            print(f"--- '{name}' ---\n")
            print(df.info(), "\n")

    return behaviors_df, history_df, articles_df

# Load data
behaviors_df, history_df, articles_df = load_data(data_version, data_type, print_info=False)

In [5]:
# Load data from parquet files
def load_data(version, data_type, print_info=False):
    base_path = f"./data_processed/{version}_{data_type}_"
    files = ["behaviors_df_expanded.parquet", "history_df_expanded.parquet", "articles_df_expanded.parquet", "users_df_expanded.parquet"]
    dataframes = [pd.read_parquet(f"{base_path}{file}") for file in files]
    
    if print_info:
        for df in dataframes:
            print(df.info(), "\n")
    
    return dataframes

# Load users_df
_, _, _, users_df = load_data(data_version, data_type, print_info=False)

In [6]:
# Filter rows where 'article_ids_clicked' has more than 1 element
behaviors_df = behaviors_df[behaviors_df['article_ids_clicked'].apply(lambda x: len(x) <= 1)]

# Convert the list to an integer value
behaviors_df['article_ids_clicked'] = behaviors_df['article_ids_clicked'].apply(lambda x: x[0] if x else None).astype('Int64')

In [6]:
# Merge history_df into behaviors_df
df = behaviors_df.merge(
    history_df,
    how='inner',
    left_on=['user_id'],
    right_on=['user_id']
)

In [7]:
# Merge articles_df into df
df = df.merge(
    articles_df.add_prefix('clicked_article_'),
    how='inner',
    left_on=['article_ids_clicked'],
    right_on=['clicked_article_article_id']
)

# Drop 'clicked_article_article_id' column
df = df.drop(columns=['clicked_article_article_id'])

In [ ]:
# Merge users_df into df
df = df.merge(
    users_df,
    how='inner',
    left_on=['user_id'],
    right_on=['user_id']
)

print(df.iloc[1000].transpose())

In [ ]:
# Create a Series mapping article_ids_clicked to topics for quick lookup
article_topics_map = df.set_index('article_ids_clicked')['clicked_article_topics'].to_dict()

# Precompute the impression times window for each article
impressions_df = df[['impression_time', 'article_ids_clicked']]

# Create a dictionary to store trendiness scores
trendiness_scores = {}

# Use tqdm to wrap the outer loop for progress tracking
for article_ids_clicked, group in tqdm(impressions_df.groupby('article_ids_clicked'), desc="Calculating Trendiness"):
    # Get the topics for the current article
    topics = article_topics_map.get(article_ids_clicked, [])
    
    # Calculate trendiness for each impression time in the group
    for impression_time in group['impression_time']:
        start_time = impression_time - pd.Timedelta(days=7)
        
        # Filter impressions in the time window
        relevant_impressions = impressions_df[
            (impressions_df['impression_time'] >= start_time) &
            (impressions_df['impression_time'] < impression_time)
        ]
        
        # Calculate trendiness by checking topic overlap
        trendiness = relevant_impressions['article_ids_clicked'].apply(
            lambda x: any(topic in article_topics_map.get(x, []) for topic in topics)
        ).sum()
        
        # Store the trendiness score
        trendiness_scores[(article_ids_clicked, impression_time)] = trendiness

# Map the trendiness scores back to the df
df['trendiness'] = df.apply(
    lambda row: trendiness_scores.get((row['article_ids_clicked'], row['impression_time']), 0), axis=1
)

In [12]:
import os

file_path = os.path.join('data_processed', f'DATA1.parquet')
df.to_parquet(file_path)

In [4]:
# Define the file path
file_path = os.path.join('data_processed', 'DATA1.parquet')

# Read the Parquet file into a DataFrame
df = pd.read_parquet(file_path)

In [ ]:
print(df[['user_id', 'article_ids_clicked']])

In [ ]:
# Precompute the impression times window for each article
impressions_df = df[['user_id', 'impression_time', 'article_ids_inview']]

# Convert 'article_ids_inview' and 'article_ids_clicked' to lists explicitly
def convert_to_list(x):
    if isinstance(x, list):
        return x
    elif isinstance(x, str):
        return x.strip('[]').replace("'", "").split(', ')
    elif isinstance(x, np.ndarray):
        return x.tolist()
    else:
        return []

df['article_ids_inview'] = df['article_ids_inview'].apply(convert_to_list)

# seen_before_clicked calculation function
def calculate_seen_before_clicked(user_id, article_id, impression_time, hours=48):
    start_time = impression_time - pd.Timedelta(hours=hours)
    relevant_impressions = impressions_df[
        (impressions_df['user_id'] == user_id) &
        (impressions_df['impression_time'] >= start_time) & 
        (impressions_df['impression_time'] < impression_time)
    ]
    seen_before_clicked = relevant_impressions['article_ids_inview'].apply(lambda x: article_id in x).sum()
    return seen_before_clicked

# Apply the function with a progress bar
df['seen_before_clicked'] = df.progress_apply(
    lambda row: calculate_seen_before_clicked(row['user_id'], row['article_id'], row['impression_time']), axis=1
)

In [ ]:
print(df[['user_id', 'article_ids_clicked', 'seen_before_clicked']])

In [8]:
# Article Delay
df['article_delay'] = (df['impression_time'] - df['clicked_article_published_time']).dt.total_seconds() / 60

In [9]:
df['impression_hour'] = df['impression_time'].dt.hour
df['impression_day_of_week'] = df['impression_time'].dt.dayofweek

# Convert time of day and day of week to cyclical features
df['impression_hour_sin'] = np.sin(2 * np.pi * df['impression_hour'] / 24)
df['impression_hour_cos'] = np.cos(2 * np.pi * df['impression_hour'] / 24)
df['impression_day_of_week_sin'] = np.sin(2 * np.pi * df['impression_day_of_week'] / 7)
df['impression_day_of_week_cos'] = np.cos(2 * np.pi * df['impression_day_of_week'] / 7)

In [ ]:
def get_ner_clusters(article_ids, articles_df):
    # Filter the articles_df to get NER clusters for the given article IDs
    clusters = articles_df[articles_df['article_id'].isin(article_ids)]['ner_clusters']
    # Flatten the list of lists and join into a single string
    return ' '.join([item for sublist in clusters for item in sublist])

# Extract NER clusters for each article_id_fixed
df['article_ner_clusters'] = df['article_id_fixed'].apply(lambda ids: get_ner_clusters(ids, articles_df))

# Convert the clicked_article_ner_clusters to strings
df['clicked_article_ner_clusters_str'] = df['clicked_article_ner_clusters'].apply(lambda x: ' '.join(x))

# Combine all NER cluster strings for TF-IDF computation
corpus = pd.concat([df['clicked_article_ner_clusters_str'], df['article_ner_clusters']])

# Create a TF-IDF vectoriser and fit it to the corpus
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(corpus)

# Calculate cosine similarity between clicked NER clusters and article NER clusters
def calculate_similarity(row_index):
    # Compute similarity between the clicked NER and article NER for each row
    return cosine_similarity(tfidf_matrix[row_index], tfidf_matrix[row_index + len(df)])[0][0]

# Apply similarity calculation for each row
df['ner_similarity'] = [calculate_similarity(i) for i in range(len(df))]

# Display the results
df[['article_ids_clicked', 'ner_similarity']]

In [15]:
import os

file_path = os.path.join('data_processed', f'DATA2.parquet')
df.to_parquet(file_path)

# -------------------------------------------------------------------------------

In [52]:
# Define the file path
file_path = os.path.join('data_processed', 'DATA2.parquet')

# Read the Parquet file into a DataFrame
df = pd.read_parquet(file_path)

In [53]:
# Ensure 'article_ids_inview' is a list
df['article_ids_inview'] = df['article_ids_inview'].apply(lambda x: list(x))

# Generate binary labels
def generate_binary_labels(row):
    return [1 if article_id == row['article_ids_clicked'] else 0 for article_id in row['article_ids_inview']]

df['binary_labels'] = df.apply(generate_binary_labels, axis=1)

In [54]:
# Define the columns to extract
columns_to_extract = [
    'article_ids_inview',
    'trendiness',
    'seen_before_clicked',
    'article_delay',
    'impression_hour_sin',
    'impression_hour_cos',
    'impression_day_of_week_sin',
    'impression_day_of_week_cos',
    'ner_similarity',
    'clicked_article_sentiment_score',
    'binary_labels'
]

# Create the new dataframe with the specified columns
df = df[columns_to_extract]

# Separate the target from the features
X = df.drop('binary_labels', axis=1)
y = df['binary_labels']

In [55]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Flatten the data
X_inview = np.concatenate(df['article_ids_inview'].values)
y = np.concatenate(df['binary_labels'].values)

# Repeat other features for each article in 'article_ids_inview'
features = [
    'trendiness', 'seen_before_clicked', 'article_delay', 
    'impression_hour_sin', 'impression_hour_cos', 
    'impression_day_of_week_sin', 'impression_day_of_week_cos', 
    'ner_similarity', 'clicked_article_sentiment_score'
]

X_other_features = np.repeat(df[features].values, df['article_ids_inview'].apply(len).values, axis=0)

# Combine 'article_ids_inview' with other features
X_combined = np.column_stack((X_inview, X_other_features))

# Standardize the features (excluding 'article_ids_inview')
scaler = StandardScaler()
X_combined[:, 1:] = scaler.fit_transform(X_combined[:, 1:])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

In [ ]:
max_article_id = max(np.concatenate(df['article_ids_inview'].values))
print(f"Maximum article ID: {max_article_id}")

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Concatenate

# Define the model
input_layer = Input(shape=(X_train.shape[1],))
embedding_layer = Embedding(input_dim=max_article_id + 1, output_dim=8, input_length=1)(input_layer[:, 0])
embedding_layer = Flatten()(embedding_layer)
concatenated = Concatenate()([embedding_layer, input_layer[:, 1:]])

x = Dense(128, activation='relu')(concatenated)
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)
output_layer = Dense(1, activation='sigmoid')(x)  # Binary classification

model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

# Calculate class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y)
class_weights_dict = dict(enumerate(class_weights))
print(f"Class weights: {class_weights_dict}")

# Fit the model with class weights
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, class_weight=class_weights_dict)

In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.4f}')

In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.4f}')